In [1]:
from transformers import GPT2LMHeadModel, GPT2TokenizerFast, AutoModelForSequenceClassification
import torch

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


In [37]:
tokenizer = AutoTokenizer.from_pretrained("gpt2")

model = AutoModelWithLMHead.from_pretrained("gpt2")

In [2]:
from transformers import AutoTokenizer, AutoModelWithLMHead

tokenizer = AutoTokenizer.from_pretrained("lvwerra/gpt2-imdb")

model = AutoModelWithLMHead.from_pretrained("lvwerra/gpt2-imdb")

/anaconda/envs/fastai2/lib/python3.8/site-packages/transformers/modeling_auto.py:821: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


In [28]:
sentiment_model = AutoModelForSequenceClassification.from_pretrained("lvwerra/bert-imdb")
sentiment_tokenizer = AutoTokenizer.from_pretrained("lvwerra/bert-imdb")

In [32]:
text = 'this movie is great'

'neg' 'pos'

In [33]:
output = sentiment_model.forward(sentiment_tokenizer.encode(text, return_tensors="pt"))
output

SequenceClassifierOutput(loss=None, logits=tensor([[-3.8915,  4.0301]], grad_fn=<AddmmBackward>), hidden_states=None, attentions=None)

In [48]:
if output.logits[0][1] > output.logits[0][1]:
    cls = 'pos'
else:
    cls = 'neg'
    
print(cls)

In [11]:
ids = tokenizer.encode('I think that this movie is terrible. In fact I think it is the worst movie I have ever seen. The acting is terrible and the plot is terrible. The only saving grace is Jackie Chan, who is a great actor. I think that this movie is a waste of time. Some part of me thinks that this movie could be saved if the romantic scenes were')

In [4]:
tokenizer.eos_token_id

50256

In [3]:
model.to('cuda')

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): Laye

In [63]:
tokenizer.eos_token_id

50256

In [104]:
model.generate??

In [11]:
def generate_text_transformer(language_model, tokenizer, text, n_words_max):
    ids = tokenizer.encode(text)
    t = torch.LongTensor(ids)[None].to('cuda')
    phrase = language_model.generate(input_ids=t, num_beams=5, temperature=1.2,  max_length=(len(ids) + 10), skip_special_tokens=True, do_sample=True)
    prediction = phrase[0][t.size(1):].cpu()
    prediction = prediction[prediction!=50256]
    
    return tokenizer.decode(prediction.cpu().numpy())

In [12]:
text = "the funny thing that"

In [19]:
generate_text_transformer(model, tokenizer, text=text, n_words_max=5)

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


' happened to me was when I was in high school'

In [180]:
def complete_word_transformer(language_model, tokenizer, text, final_word):
    if len(text) == 0:
        return ''
    ids = tokenizer.encode(text)
    t = torch.LongTensor(ids)[None].to('cuda')
    logits = model.forward(t).logits[0][-1]
    sorted_indices = torch.argsort(logits, descending=True)
    
    for tk_idx in sorted_indices:
        word = tokenizer.decode(tk_idx.cpu().numpy())
        if word.lower().startswith(final_word):
            print('here')
            print(word)
            return word[len(final_word):]
    return ""
#     sorted_tokens = [tokenizer.decode[i.cpu().numpy()] for i in sorted_indices]
#     
#     sorted_tokens = []
#     for tk in sorted_indices:
#         sorted_tokens.append(1)
#     sorted_tokens.append(tokenizer.decode([123]))
        
#     candidate_tokens = [token for token in sorted_tokens if token.lower().startswith(final_word)]
#     phrase = language_model.generate(t, max_length=(len(ids) + n_words_max))
#     prediction = phrase[0][t.size(1):]
#     return tokenizer.decode(prediction.cpu().numpy(), skip_special_tokens=True)

In [181]:
complete_word_transformer(model, tokenizer, text='the', final_word='re')

here
re


''

In [103]:
model.forward(t).logits[0][-1].size()

torch.Size([50257])

In [86]:
def greedy_search(
    language_model,
    input_ids: torch.LongTensor,
    logits_processor: Optional[LogitsProcessorList] = None,
    max_length: Optional[int] = None,
    pad_token_id: Optional[int] = None,
    eos_token_id: Optional[int] = None,
    **model_kwargs
):


    # init values
    logits_processor = logits_processor if logits_processor is not None else LogitsProcessorList()
    max_length = max_length if max_length is not None else language_model.config.max_length
    pad_token_id = pad_token_id if pad_token_id is not None else language_model.config.pad_token_id
    eos_token_id = eos_token_id if eos_token_id is not None else language_model.config.eos_token_id

    # init sequence length tensors
    sequence_lengths, unfinished_sequences, cur_len = language_model._init_sequence_length_for_generation(
        input_ids, max_length
    )

    while cur_len < max_length:
        # prepare model inputs
        model_inputs = language_model.prepare_inputs_for_generation(input_ids, **model_kwargs)

        # forward pass to get next token
        outputs = language_model(**model_inputs, return_dict=True)
        next_token_logits = outputs.logits[:, -1, :]
        print(next_token_logits.size())
        # pre-process distribution
        scores = logits_processor(input_ids, next_token_logits)

        # argmax
        next_tokens = torch.argmax(scores, dim=-1)

        # add code that transfomers next_tokens to tokens_to_add
        if eos_token_id is not None:
            assert pad_token_id is not None, "If eos_token_id is defined, make sure that pad_token_id is defined."
            next_tokens = next_tokens * unfinished_sequences + (pad_token_id) * (1 - unfinished_sequences)

        # add token and increase length by one
        input_ids = torch.cat([input_ids, next_tokens[:, None]], dim=-1)

        # update sequence length
        if eos_token_id is not None:
            sequence_lengths, unfinished_sequences = language_model._update_seq_length_for_generation(
                sequence_lengths, unfinished_sequences, cur_len, next_tokens == eos_token_id
            )

        # update model kwargs
        model_kwargs = language_model._update_model_kwargs_for_generation(
            outputs, model_kwargs, is_encoder_decoder=language_model.config.is_encoder_decoder
        )

        # stop when there is a </s> in each sentence, or if we exceed the maximul length
        if unfinished_sequences.max() == 0:
            break

        # increase cur_len
        cur_len = cur_len + 1

    return input_ids

NameError: name 'Optional' is not defined

Objective ->

1. Query Text -> [word_complete_api] -> Query Text + remaining portion of the word 
2. After remaining portion of the word is returned -> [phrase_complete_api] -> " " + remaining portion of the phrase


if event.target.value == " ", suggestedText = phrase_complete output
else, suggestedText = " " + phrase_complete_output